In [ ]:
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

import client as cp
import middleware as mp

# Matplot lib settings
%matplotlib notebook
matplotlib.rcParams.update({
        'font.size': 11,
        'font.family': 'sans-serif',
        'xtick.labelsize': 10,
        'ytick.labelsize': 10
    })

# Numpy settings
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [ ]:
base_dir = '/Users/ruifengxu/Development/asl-fall16-project/logs/maximum-throughput/'
client_log_template = 'thread-{}-client-{}/repetition-{}-client-{}.log'
middleware_log_template = 'thread-{}-client-{}/repetition-{}-trace.log'
colors = ['mediumturquoise', 'red', 'green', 'saddlebrown', 'royalblue', 'darkmagenta', 'orange']
markers = ['^', 'v', '<', 'o', 's', '*', 'd', '>']

In [ ]:
# Root mean square form
def parse_client_entries_1(thread_pool_size_config, 
                           num_clients_config, 
                           num_repetitions, 
                           num_client_machines, 
                           offset, 
                           duration):    
    print('{:>3} | {:>3} | {:>10} | {:>10} | {:>10} | {:>10} | {:>10} | {:>10} | {:>10}'.format(
            '# T', '# C', 'AvgTP', 'StdTP', 'CITP', 'AvgRT', 'StdRT', 'CIRT', 'IRCheck'))
    print()
    
    all_tp_mean = []
    all_tp_std = []
    all_tp_ci = []
    
    all_rt_mean = []
    all_rt_std = []
    all_rt_ci = []
    
    start = offset
    end = start + duration

    for thread_pool_size in thread_pool_size_config:
        thread_tp_mean = []
        thread_tp_std = []
        thread_tp_ci = []

        thread_rt_mean = []
        thread_rt_std = []
        thread_rt_ci = []
        
        for num_clients in num_clients_config:
            client_tps = np.zeros(num_repetitions)
            client_rts = np.zeros(num_repetitions)
            client_rts_var = np.zeros(num_repetitions)
            
            for repetition in range(1, num_repetitions + 1, 1):
                repetition_tp = 0
                repetition_rts = np.zeros(duration * num_client_machines)
                repetition_rts_weight = np.zeros(duration * num_client_machines)
                repetition_rts_index = 0
                repetition_rts_var = 0.0
                        
                for log_id in range(1, num_client_machines + 1, 1):
                    filename = os.path.join(
                        base_dir,
                        client_log_template.format(thread_pool_size, num_clients, repetition, log_id)
                    )
                    data = cp.parse_log(filename)
                    
                    num_ops = data['get_global'][end - 2]['num_ops'] - data['get_global'][start - 2]['num_ops']
                    
                    repetition_tp = repetition_tp + (num_ops // duration)
                    
                    for entry in data['get_local'][start - 1: end - 1]:
                        repetition_rts[repetition_rts_index] =  entry['rt_mean']
                        repetition_rts_weight[repetition_rts_index] = entry['num_ops']
                        repetition_rts_index = repetition_rts_index + 1                        
                        repetition_rts_var = repetition_rts_var + float(
                            entry['rt_std'] * entry['rt_std'] * (entry['num_ops'] - 1))
                
                client_tps[repetition - 1] = repetition_tp
                client_rts[repetition - 1] = np.average(repetition_rts, weights=repetition_rts_weight)
                client_rts_var[repetition - 1] = repetition_rts_var / (np.sum(repetition_rts_weight) - duration)
            
            client_tp_mean = np.mean(client_tps)
            client_tp_std = np.std(client_tps, ddof=1)
            client_tp_ci = np.floor(2.776 * client_tp_std / np.sqrt(num_repetitions))
            
            client_rt_mean = np.mean(client_rts)
            client_rt_std = np.sqrt(np.sum(client_rts_var) / num_repetitions)
            client_rt_ci = 2.776 * client_rt_std / np.sqrt(num_repetitions)
            
            print('{:3} | {:3} | {:10.0f} | {:10.0f} | {:10.0f} | {:10.3f} | {:10.3f} | {:10.3f} | {:10.3f}'.format(
                    thread_pool_size, num_clients, 
                    client_tp_mean, client_tp_std, client_tp_ci,
                    client_rt_mean, client_rt_std, client_rt_ci,
                    1 / client_rt_mean * 1000 * 3 * num_clients / client_tp_mean))
            
            thread_tp_mean.append(client_tp_mean)
            thread_tp_std.append(client_tp_std)
            thread_tp_ci.append(client_tp_ci)

            thread_rt_mean.append(client_rt_mean)
            thread_rt_std.append(client_rt_std)
            thread_rt_ci.append(client_rt_ci)   
        
        print()
        
        all_tp_mean.append(thread_tp_mean)
        all_tp_std.append(thread_tp_std)
        all_tp_ci.append(thread_tp_ci)
        
        all_rt_mean.append(thread_rt_mean)
        all_rt_std.append(thread_rt_std)
        all_rt_ci.append(thread_rt_ci)
    
    return {
        'tp': {
            'mean': all_tp_mean,
            'std': all_tp_std,
            'ci': all_tp_ci
        },
        'rt': {
            'mean': all_rt_mean,
            'std': all_rt_std,
            'ci': all_rt_ci
        }
    }

In [ ]:
# Pooled variance formulation
def parse_client_entries_2(thread_pool_size_config, 
                           num_clients_config, 
                           num_repetitions, 
                           num_client_machines, 
                           offset, 
                           duration):
    print('{:>3} {:>9} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(
            'T', 'C', 'AvgTP', 'StdTP', 'CITP', 'AvgRT', 'StdRT', 'CIRT', 'IRCheck'))
    print()

    all_tp_mean = []
    all_tp_std = []
    all_tp_ci = []

    all_rt_mean = []
    all_rt_std = []
    all_rt_ci = []

    start = offset
    end = start + duration
        
    for thread_pool_size in thread_pool_size_config:
        thread_tp_mean = []
        thread_tp_std = []
        thread_tp_ci = []
        
        thread_rt_mean = []
        thread_rt_std = []
        thread_rt_ci = []
        
        for num_clients in num_clients_config:
            client_tps = np.zeros(num_repetitions)
            
            client_rts = np.zeros(num_repetitions * num_client_machines * duration)
            client_rts_weight = np.zeros(num_repetitions * num_client_machines * duration)
            client_rts_index = 0
            client_rts_var = 0.0

            for repetition in range(1, num_repetitions + 1, 1):
                repetition_tp = 0
                
                for log_id in range(1, num_client_machines + 1, 1):
                    filename = os.path.join(
                        base_dir,
                        client_log_template.format(thread_pool_size, num_clients, repetition, log_id)
                    )
                    data = cp.parse_log(filename)
                    
                    num_ops = data['get_global'][end - 2]['num_ops'] - data['get_global'][start - 2]['num_ops']
                    
                    repetition_tp = repetition_tp + (num_ops // duration)
                    
                    for entry in data['get_local'][start - 1: end - 1]:
                        client_rts[client_rts_index] = entry['rt_mean']
                        client_rts_weight[client_rts_index] = entry['num_ops']
                        client_rts_index = client_rts_index + 1
                        client_rts_var = client_rts_var + float(
                            entry['rt_std'] * entry['rt_std'] * (entry['num_ops'] - 1))
                
                client_tps[repetition - 1] = repetition_tp
            
            client_tp_mean = np.floor(np.mean(client_tps))
            client_tp_std = np.floor(np.std(client_tps, ddof=1))
            client_tp_ci = np.floor(2.776 * client_tp_std / np.sqrt(num_repetitions))

            client_rt_mean = np.average(client_rts, weights=client_rts_weight)
            client_rt_std = np.sqrt(
                client_rts_var / 
                (np.sum(client_rts_weight) - (duration * num_repetitions * num_client_machines))
            )
            client_rt_ci = 1.960 * client_rt_std / np.sqrt(duration * num_repetitions * num_client_machines)
            
            print('{:3} {:3} ({:3}) {:10.0f} {:10.0f} {:10.0f} {:10.3f} {:10.3f} {:10.3f} {:10.3f}'.format(
                    thread_pool_size, num_clients, num_clients * num_client_machines,
                    client_tp_mean, client_tp_std, client_tp_ci,
                    client_rt_mean, client_rt_std, client_rt_ci,
                    1 / client_rt_mean * 1000 * num_clients * num_client_machines / client_tp_mean))
        
            thread_tp_mean.append(client_tp_mean)
            thread_tp_std.append(client_tp_std)
            thread_tp_ci.append(client_tp_ci)

            thread_rt_mean.append(client_rt_mean)
            thread_rt_std.append(client_rt_std)
            thread_rt_ci.append(client_rt_ci)  
        
        print()

        all_tp_mean.append(thread_tp_mean)
        all_tp_std.append(thread_tp_std)
        all_tp_ci.append(thread_tp_ci)
        
        all_rt_mean.append(thread_rt_mean)
        all_rt_std.append(thread_rt_std)
        all_rt_ci.append(thread_rt_ci)
    
    return {
        'tp': {
            'mean': all_tp_mean,
            'std': all_tp_std,
            'ci': all_tp_ci
        },
        'rt': {
            'mean': all_rt_mean,
            'std': all_rt_std,
            'ci': all_rt_ci
        }
    }

In [ ]:
# Percentile variation
def parse_client_entries_3(thread_pool_size_config, 
                           num_clients_config, 
                           num_repetitions, 
                           num_client_machines, 
                           offset, 
                           duration):
    print('{:>3} {:>9} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10} {:>10}'.format(
            'T', 'C', 'AvgTP', 'StdTP', 'CITP', 'AvgRT', 'Low', 'High', 'IRCheck'))
    print()

    all_tp_mean = []
    all_tp_std = []
    all_tp_ci = []

    all_rt_mean = []
    all_rt_plow = []
    all_rt_phigh = []

    start = offset
    end = start + duration
    
    for thread_pool_size in thread_pool_size_config:
        thread_tp_mean = []
        thread_tp_std = []
        thread_tp_ci = []
        
        thread_rt_mean = []
        thread_rt_plow = []
        thread_rt_phigh = []
                
        for num_clients in num_clients_config:
            client_tps = np.zeros(num_repetitions)
            
            client_rts = np.zeros(num_repetitions * num_client_machines * duration)
            client_rts_weight = np.zeros(num_repetitions * num_client_machines * duration)
            client_rts_index = 0
            client_rts_bucket = None

            for repetition in range(1, num_repetitions + 1, 1):
                repetition_tp = 0
                
                for log_id in range(1, num_client_machines + 1, 1):
                    filename = os.path.join(
                        base_dir,
                        client_log_template.format(thread_pool_size, num_clients, repetition, log_id)
                    )
                    data = cp.parse_log(filename)
                    
                    num_ops = data['get_global'][end - 2]['num_ops'] - data['get_global'][start - 2]['num_ops']
                    
                    repetition_tp = repetition_tp + (num_ops // duration)
                    
                    for entry in data['get_local'][start - 1: end - 1]:
                        client_rts[client_rts_index] = entry['rt_mean']
                        client_rts_weight[client_rts_index] = entry['num_ops']
                        client_rts_index = client_rts_index + 1
                        
                    if client_rts_bucket is None:
                        client_rts_bucket = list(data['get_bucket'])
                    else:
                        client_rts_bucket = [x + y for x, y in zip(client_rts_bucket, data['get_bucket'])]     
                        
                client_tps[repetition - 1] = repetition_tp
            client_tp_mean = np.floor(np.mean(client_tps))
            client_tp_std = np.floor(np.std(client_tps, ddof=1))
            client_tp_ci = np.floor(2.776 * client_tp_std / np.sqrt(num_repetitions))

            client_rt_mean = np.average(client_rts, weights=client_rts_weight)
            client_rt_plow = cp.get_percentile(client_rts_bucket, 5)
            client_rt_phigh = cp.get_percentile(client_rts_bucket, 95)
            
            print('{:3} {:3} ({:3}) {:10.0f} {:10.0f} {:10.0f} {:10.3f} {:10.3f} {:10.3f} {:10.3f}'.format(
                    thread_pool_size, num_clients, num_clients * num_client_machines,
                    client_tp_mean, client_tp_std, client_tp_ci,
                    client_rt_mean, client_rt_plow, client_rt_phigh,
                    1 / client_rt_mean * 1000 * num_clients * num_client_machines / client_tp_mean))
        
            thread_tp_mean.append(client_tp_mean)
            thread_tp_std.append(client_tp_std)
            thread_tp_ci.append(client_tp_ci)

            thread_rt_mean.append(client_rt_mean)
            thread_rt_plow.append(client_rt_plow)
            thread_rt_phigh.append(client_rt_phigh)  
        
        print() 
        
        print()

        all_tp_mean.append(thread_tp_mean)
        all_tp_std.append(thread_tp_std)
        all_tp_ci.append(thread_tp_ci)
        
        all_rt_mean.append(thread_rt_mean)
        all_rt_plow.append(thread_rt_plow)
        all_rt_phigh.append(thread_rt_phigh)
    
    return {
        'tp': {
            'mean': all_tp_mean,
            'std': all_tp_std,
            'ci': all_tp_ci
        },
        'rt': {
            'mean': all_rt_mean,
            'plow': all_rt_plow,
            'phigh': all_rt_phigh
        }
    }

In [ ]:
def check_middleware_run_times(thread_pool_size_config, 
                               num_client_config,
                               num_repetitions,
                               request_type):
    run_times = []
    for thread_pool_size in thread_pool_size_config:
        for num_clients in num_client_config:
            print('{:3} {:3}: '.format(thread_pool_size, num_clients), end='')
            for repetition in range(1, num_repetitions + 1, 1):            
                filename = os.path.join(
                        base_dir,
                        middleware_log_template.format(thread_pool_size, num_clients, repetition))
                middleware_data = mp.parse_log(filename)
                run_time = mp.count_runtime(middleware_data, request_type=request_type)
                print('{:3}s'.format(run_time), end=' ')
                run_times.append(run_time)
            print()
        print()
    fig = plt.figure(figsize=(5, 4))
    plt.hist(run_times, bins=np.arange(min(run_times), max(run_times) + 1, 1), alpha=0.7, facecolor='green')
    plt.xlabel('Total middleware runtime (s)')
    plt.xticks(np.arange(min(run_times), max(run_times) + 2, 1))
    plt.ylim([0, 100])
    plt.yticks(np.arange(0, 100, 10))
    plt.ylabel('Number of experiments')
    ax = plt.gca()
    ax.grid(True)
    plt.tight_layout()

In [ ]:
check_middleware_run_times(
    [4, 8, 12, 16, 20, 24],
    [10, 40, 70, 100, 130, 160, 190], 
    5,
    'g')

In [ ]:
t = 0
for i, r in enumerate(client_data['tp']['mean']):
    if i == 0:
        print((r[1] - r[0]) / 1, i)
    else:
        t = (r[1] - r[0]) / 1 + t
    
print(t / 5)

t = 0
for i, r in enumerate(client_data['rt']['mean']):
    if i == 0:
        print((r[1] - r[0]) / 1, i)
    else:
        t = (r[1] - r[0]) / 1 + t
    
print(t / 5)

print()

t = 0
for i, r in enumerate(client_data['tp']['mean']):
    if i <= 1:
        print((r[2] - r[1]) / 1, i)
    else:
        t = (r[2] - r[1]) / 1 + t
    
print(t / 4)

t = 0
for i, r in enumerate(client_data['rt']['mean']):
    if i <= 1:
        print((r[3] - r[2]) / 1, i)
    else:
        t = (r[3] - r[2]) / 1 + t
    
print(t / 4)

print()

t = 0
for i, r in enumerate(client_data['tp']['mean']):
    if i <= 1:
        print((r[3] - r[2]) / 1, i)
    else:
        t = (r[3] - r[2]) / 1 + t
    
print(t / 4)

t = 0
for i, r in enumerate(client_data['rt']['mean']):
    if i <= 1:
        print((r[3] - r[2]) / 1, i)
    else:
        t = (r[3] - r[2]) / 1 + t
    
print(t / 4)

print()

j = 3
for i, r in enumerate(client_data['tp']['mean']):
    print(r[j] - r[j - 1])

for i, r in enumerate(client_data['rt']['mean']):
    print(r[j] - r[j - 1])


In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.gca(projection='3d')
X = np.arange(30, 600, 90)
Y = np.arange(4, 28, 4)
X, Y = np.meshgrid(X, Y)
Z = np.array(client_data['tp']['mean'])
surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap=cm.YlOrRd,
                       alpha=0.8, antialiased=True, linewidth=0.8)
ax.set_xlabel('Number of clients')
ax.set_xticks(np.insert(np.arange(30, 600, 90), 0, 0))
ax.set_ylabel('Thread pool size')
ax.set_yticks(np.arange(0, 28, 4))
ax.set_zlim(0, 25000)
ax.set_zticks(np.arange(0, 27500, 2500))
ax.zaxis.set_rotate_label(False)
ax.set_zlabel('Aggregated throughput (request/s)', rotation=90)
ax.view_init(10, 223)

fig.colorbar(surf, shrink=0.5, aspect=15)

plt.tight_layout()

fig.savefig('thread-client-3d.png', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
fig = plt.figure(figsize=(12, 6))
labels = ['4 threads', '8 threads', '12 threads', '16 threads', '20 threads', '24 threads']

# Subplot
plt.subplot(1, 2, 1)
for index, trend_tp_mean in enumerate(client_data['tp']['mean']):
    plt.errorbar(np.arange(30, 600, 90), trend_tp_mean, yerr=client_data['tp']['ci'][index],
                 alpha=1.0, color=colors[index], label=labels[index],
                 capsize=5, elinewidth=0.5, 
                 markeredgewidth=0.8,
                 linestyle='-', linewidth=1.5)
plt.xlim([0, 600])
plt.xticks(np.insert(np.arange(30, 600, 90), 0, 0))
plt.xlabel('Number of clients')
plt.ylim([0, 27500])
plt.yticks(np.arange(0, 27500, 2500))
plt.ylabel('Aggregated throughput (requests/s)')
plt.title('(a)')

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

# Subplot
plt.subplot(1, 2, 2)
for index, thread_rt_mean in enumerate(client_data['rt']['mean']):
    plt.errorbar(np.arange(30, 600, 90), thread_rt_mean, 
                 yerr=[
                    np.array(thread_rt_mean) - np.array(client_data['rt']['plow'][index]), 
                    np.array(client_data['rt']['phigh'][index]) - np.array(thread_rt_mean)
                 ],
                 alpha=1.0, color=colors[index], label=labels[index],
                 capsize=5, elinewidth=0.5, 
                 markeredgewidth=0.8,
                 linestyle='-', linewidth=1.5)
plt.xlim([0, 600])
plt.xticks(np.insert(np.arange(30, 600, 90), 0, 0))
plt.xlabel('Number of clients')
plt.ylim([0, 110])
plt.yticks(np.arange(0, 110, 10))
plt.ylabel('Average response time (ms)')
plt.title('(b)')
    
ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

fig.savefig('thread-throughput-response-time.png', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
fig = plt.figure(figsize=(12, 6))
labels = ['30 clients', '120 clients', '210 clients', '300 clients', '390 clients', '480 clients', '570 clients']


# Subplot
plt.subplot(1, 2, 1)
for index in range(0, len(labels), 1):
    y = [thread_tp_mean[index] for thread_tp_mean in client_data['tp']['mean']] 
    yerr = [thread_tp_ci[index] for thread_tp_ci in client_data['tp']['ci']] 
    plt.errorbar(np.arange(4, 28, 4), y, yerr=yerr,
                 alpha=1.0, color=colors[index], label=labels[index],
                 capsize=5, elinewidth=0.5,
                 markeredgewidth=0.8,
                 linestyle='-', linewidth=1.5)
    plt.xlim([0, 28])
    plt.xticks(np.arange(0, 28, 4))
    plt.xlabel('Number of threads')
    plt.ylim([0, 27500])
    plt.yticks(np.arange(0, 27500, 2500))
    plt.ylabel('Aggregated throughput (requests/s)')
    plt.title('(a)')

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=4, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

# Subplot
plt.subplot(1, 2, 2)
for index in range(0, len(labels), 1):
    y = [thread_rt_mean[index] for thread_rt_mean in client_data['rt']['mean']] 
    yerr=[
        np.array(y) - np.array([thread_rt_plow[index] for thread_rt_plow in client_data['rt']['plow']]), 
        np.array([thread_rt_phigh[index] for thread_rt_phigh in client_data['rt']['phigh']]) - np.array(y)
    ]
    plt.errorbar(np.arange(4, 28, 4), y, 
                 yerr=yerr,
                 alpha=1.0, color=colors[index], label=labels[index],
                 capsize=5, elinewidth=0.5,
                 markeredgewidth=0.8,
                 linestyle='-', linewidth=1.5)
    
    plt.xlim([0, 28])
    plt.xticks(np.arange(0, 28, 4))
    plt.xlabel('Thread pool size')
    plt.ylim([0, 110])
    plt.yticks(np.arange(0, 110, 10))
    plt.ylabel('Average response time (ms)')
    plt.title('(b)')

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=4, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

fig.savefig('client-throughput-response-time.png', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
fig = plt.figure(figsize=(9, 6))
titles = ['(a) 4 threads', '(b) 8 threads', '(c) 12 threads', '(d) 16 threads', '(e) 20 threads', '(f) 24 threads']

for index, thread_tp_mean in enumerate(client_data['tp']['mean']):
    plt.subplot(2, 3, index + 1)
    plt.errorbar(np.arange(30, 600, 90), thread_tp_mean, yerr=client_data['tp']['ci'][index], color=colors[index])
    plt.xlim([0, 600])
    plt.xticks(np.insert(np.arange(30, 600, 90), 0, 0))
    if index >= 3:
        plt.xlabel('Number of clients')
    plt.ylim([0, 25000])
    plt.yticks(np.arange(0, 25001, 2500))
    if index % 3 == 0:
        plt.ylabel('Aggregated throughput (requests/s)')
    plt.title(titles[index], fontsize=11)

    ax = plt.gca()
    ax.grid(True)

plt.tight_layout()

In [ ]:
fig = plt.figure(figsize=(9, 6))
titles = ['(a) 4 threads', '(b) 8 threads', '(c) 12 threads', '(d) 16 threads', '(e) 20 threads', '(f) 24 threads']

for index, thread_rt_mean in enumerate(client_data['rt']['mean']):
    plt.subplot(2, 3, index + 1)
    plt.errorbar(np.arange(30, 600, 90), thread_rt_mean, 
                 yerr=[
                    np.array(thread_rt_mean) - np.array(client_data['rt']['plow'][index]), 
                    np.array(client_data['rt']['phigh'][index]) - np.array(thread_rt_mean)
                 ],
                 color=colors[index])
    plt.xlim([0, 600])
    plt.xticks(np.insert(np.arange(30, 600, 90), 0, 0))
    if index >= 3:
        plt.xlabel('Number of clients')
    plt.ylim([0, 100])
    plt.yticks(np.arange(0, 101, 10))
    if index % 3 == 0:
        plt.ylabel('Average response time (ms)')
    plt.title(titles[index], fontsize=11)

    ax = plt.gca()
    ax.grid(True)

plt.tight_layout()

In [ ]:
check_middleware_run_times(
    [12, 16, 20], 
    [4, 7, 10, 20, 30, 40, 50, 60, 80, 90, 100, 110, 120, 130, 140, 150],
    5,
    'g')

In [ ]:
client_data_fine = parse_client_entries_3(
    [12, 16, 20], 
    [4, 7, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150],
    5, 
    3, 
    16, 
    30
)

In [ ]:
fig = plt.figure(figsize=(12, 6))
labels = ['12 threads', '16 threads', '20 threads']

# Subplot
plt.subplot(1, 2, 1)
x = [12, 21, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 390, 420, 450]
for index, thread_tp_mean in enumerate(client_data_fine['tp']['mean']):
    plt.errorbar(x, thread_tp_mean, yerr=client_data_fine['tp']['ci'][index],
                 alpha=1.0, color=colors[index], label=labels[index],
                 capsize=5, elinewidth=0.5, 
                 markeredgewidth=0.8,
                 linestyle='-', linewidth=1.5)
    plt.xlim([0, 480])
    plt.xticks(np.arange(0, 480, 30))
    plt.xlabel('Number of clients')
    plt.ylim([0, 25000])
    plt.yticks(np.arange(0, 25000, 2500))
    plt.ylabel('Aggregated throughput (requests/s)')

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1.02, 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

# Subplot
plt.subplot(1, 2, 2)
for index, thread_rt_mean in enumerate(client_data_fine['rt']['mean']):
    plt.errorbar(x, thread_rt_mean, 
                 yerr=[
                    np.array(thread_rt_mean) - np.array(client_data_fine['rt']['plow'][index]), 
                    np.array(client_data_fine['rt']['phigh'][index]) - np.array(thread_rt_mean)
                 ],
                 alpha=1.0, color=colors[index], label=labels[index],
                 capsize=5, elinewidth=0.5, 
                 markeredgewidth=0.8,
                 linestyle='-', linewidth=1.5)
    plt.xlim([0, 480])
    plt.xticks(np.arange(0, 480, 30))
    plt.xlabel('Number of clients')
    plt.ylim([0, 110])
    plt.yticks(np.arange(0, 110, 10))
    plt.ylabel('Average response time (ms)')

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1.02, 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

fig.savefig('thread-throughput-response-time-fine.png', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
def calculate_middleware_time_percentiles(thread_pool_size, 
                                          num_clients, 
                                          num_repetitions,
                                          percentile_config,
                                          offset, 
                                          duration):
    read_request_times = []
    queue_times = []
    send_request_times = []
    read_response_times = []
    send_response_times = []
    total_times = []

    for repetition in range(1, num_repetitions + 1, 1):                
        filename = os.path.join(
            base_dir,
            middleware_log_template.format(thread_pool_size, num_clients, repetition)
        )
        data = mp.parse_log(filename, request_type='g')
        start_time = data[0][2] + offset
        end_time = start_time + duration
        for entry in data:                                                
            if entry[2] < start_time:
                continue
            if entry[2] == end_time:
                break
            if entry[6] is False:
                continue

            read_request_times.append(entry[8])
            queue_times.append(entry[9])
            send_request_times.append(entry[10])
            read_response_times.append(entry[11])
            send_response_times.append(entry[12])
            total_times.append(np.sum(entry[8: 13])) 

    return {
        'read_request': np.percentile(read_request_times, percentile_config) / 1000000,
        'queue': np.percentile(queue_times, percentile_config) / 1000000,
        'send_request': np.percentile(send_request_times, percentile_config) / 1000000,
        'read_response': np.percentile(read_response_times, percentile_config) / 1000000,
        'send_respones': np.percentile(send_response_times, percentile_config) / 1000000,
        'total': np.percentile(total_times, percentile_config) / 1000000
    }

In [ ]:
def calculate_middleware_time_mean(thread_pool_size, 
                                   num_clients, 
                                   num_repetitions,
                                   offset, 
                                   duration):
    read_request_times = []
    queue_times = []
    send_request_times = []
    read_response_times = []
    send_response_times = []
    total_times = []
        
    for repetition in range(1, num_repetitions + 1, 1):
        filename = os.path.join(
            base_dir,
            middleware_log_template.format(thread_pool_size, num_clients, repetition)
        )
        data = mp.parse_log(filename, request_type='g')
        start_time = data[0][2] + offset
        end_time = start_time + duration
        for entry in data:                                                
            if entry[2] < start_time:
                continue
            if entry[2] == end_time:
                break
            if entry[6] is False:
                continue
            read_request_times.append(entry[8])
            queue_times.append(entry[9])
            send_request_times.append(entry[10])
            read_response_times.append(entry[11])
            send_response_times.append(entry[12])
            total_times.append(np.sum(entry[8:13])) 
    
    return {
        'read_request': [
            np.mean(read_request_times) / 1000000, 
            np.std(read_request_times) / 1000000, 
            1.960 * np.std(read_request_times) / np.sqrt(len(read_request_times)) / 1000000
        ],
        'queue': [
            np.mean(queue_times) / 1000000, 
            np.std(queue_times) / 1000000,
            1.960 * np.std(queue_times) / np.sqrt(len(queue_times)) / 1000000
        ],
        'send_request': [
            np.mean(send_request_times) / 1000000, 
            np.std(send_request_times) / 1000000,
            1.960 * np.std(send_request_times) / np.sqrt(len(send_request_times)) / 1000000
        ],
        'read_response': [
            np.mean(read_response_times) / 1000000, 
            np.std(read_response_times) / 1000000, 
            1.960 * np.std(read_response_times) / np.sqrt(len(read_response_times)) / 1000000
        ],
        'send_respones': [
            np.mean(send_response_times) / 1000000, 
            np.std(send_response_times) / 1000000,
            1.960 * np.std(send_response_times) / np.sqrt(len(send_response_times)) / 1000000
        ],
        'total': [
            np.mean(total_times) / 1000000, 
            np.std(total_times) / 1000000,
            1.960 * np.std(total_times) / np.sqrt(len(total_times)) / 1000000
        ]
    }

In [ ]:
def make_middleware_time_boxplot(thread_pool_size, 
                                 num_clients, 
                                 num_repetitions,
                                 offset, 
                                 duration):
    read_request_times = []
    queue_times = []
    send_request_times = []
    read_response_times = []
    send_response_times = []
    total_times = []
        
    for repetition in range(1, num_repetitions + 1, 1):
        filename = os.path.join(
            base_dir,
            middleware_log_template.format(thread_pool_size, num_clients, repetition)
        )
        data = mp.parse_log(filename, request_type='g')
        start_time = data[0][2] + offset
        end_time = start_time + duration
        for entry in data:                                                
            if entry[2] < start_time:
                continue
            if entry[2] == end_time:
                break
            if entry[6] is False:
                continue
            read_request_times.append(entry[8])
            queue_times.append(entry[9])
            send_request_times.append(entry[10])
            read_response_times.append(entry[11])
            send_response_times.append(entry[12])
            total_times.append(np.sum(entry[8:13])) 

    fig = plt.figure(figsize=(5, 5))
    plt.boxplot([
            np.array(queue_times) / 1000000, 
            np.array(read_response_times) / 1000000, 
            np.array(total_times) / 1000000,
        ],
        showmeans=True,
        sym='',
        whis=[0, 95]
    )
    plt.xticks([1, 2, 3], ['Queue', 'Sever', 'Total'])
    plt.ylim([0, 22])
    plt.yticks(np.arange(0, 24, 2))
    plt.ylabel('Time (ms)')
    ax = plt.gca()
    ax.yaxis.grid(True) 
    fig.savefig('middleware-boxplot.png', dpi=300, bbox_inches='tight', pad_inches=0.2)

In [ ]:
middleware_percentile = calculate_middleware_time_percentiles(
    16,
    110,
    5,
    [0, 25, 50, 75, 95, 99],
    16,
    45
)

In [ ]:
middleware_percentile

In [ ]:
middleware_mean

In [ ]:
middleware_mean = calculate_middleware_time_mean(
    16,
    110,
    5,
    16,
    45
)

In [ ]:
make_middleware_time_boxplot(
    16,
    110,
    5,
    16,
    30
)

In [ ]:
fig = plt.figure(figsize=(9, 6))
labels = ['25th', '50th', '75th', '95th']

x = [30, 120, 210, 300, 390, 480, 570]
thread_percentiles = middleware_percentiles[2]

plt.subplot(2, 3, 1)
for index in range(0, len(labels), 1):
    y = [client_percentiles[index] for client_percentiles in thread_percentiles['read_request']]
    plt.plot(x, y,
             alpha=0.7, color=colors[index], label=labels[index],
             linestyle='-', linewidth=1,
             marker=markers[index], markeredgecolor=colors[index], markeredgewidth=1.2, markersize=5)
    plt.xlim([0, 600])
    plt.xticks(x)    
    plt.xlabel('Number of clients')
    plt.ylim([0, 0.13])
    plt.yticks(np.arange(0, 0.11, 0.02))
    plt.ylabel('Time (ms)')
    plt.title('(a) Read request', fontsize=11)

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

plt.subplot(2, 3, 2)
for index in range(0, len(labels), 1):
    y = [client_percentiles[index] for client_percentiles in thread_percentiles['queue']]
    plt.plot(x, y,
             alpha=0.7, color=colors[index], label=labels[index],
             linestyle='-', linewidth=1,
             marker=markers[index], markeredgecolor=colors[index], markeredgewidth=1.2, markersize=5)
    plt.xlim([0, 600])
    plt.xticks(x)    
    plt.xlabel('Number of clients')
    plt.ylim([0, 100])
    plt.yticks(np.arange(0, 100, 10))
    plt.ylabel('Time (ms)')
    plt.title('(b) Queue', fontsize=11)

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

plt.subplot(2, 3, 3)
for index in range(0, len(labels), 1):
    y = [client_percentiles[index] for client_percentiles in thread_percentiles['send_request']]
    plt.plot(x, y,
             alpha=0.7, color=colors[index], label=labels[index],
             linestyle='-', linewidth=1,
             marker=markers[index], markeredgecolor=colors[index], markeredgewidth=1.2, markersize=5)
    plt.xlim([0, 600])
    plt.xticks(x)    
    plt.xlabel('Number of clients')
    plt.ylim([0, 0.13])
    plt.yticks(np.arange(0, 0.11, 0.02))
    plt.ylabel('Time (ms)')
    plt.title('(c) Forward request', fontsize=11)

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

plt.subplot(2, 3, 4)
for index in range(0, len(labels), 1):
    y = [client_percentiles[index] for client_percentiles in thread_percentiles['read_response']]
    plt.plot(x, y,
             alpha=0.7, color=colors[index], label=labels[index],
             linestyle='-', linewidth=1,
             marker=markers[index], markeredgecolor=colors[index], markeredgewidth=1.2, markersize=5)
    plt.xlim([0, 600])
    plt.xticks(x)    
    plt.xlabel('Number of clients')
    plt.ylim([0, 13])
    plt.yticks(np.arange(0, 11, 1))
    plt.ylabel('Time (ms)')
    plt.title('(d) Read response', fontsize=11)

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

plt.subplot(2, 3, 5)
for index in range(0, len(labels), 1):
    y = [client_percentiles[index] for client_percentiles in thread_percentiles['send_response']]
    plt.plot(x, y,
             alpha=0.7, color=colors[index], label=labels[index],
             linestyle='-', linewidth=1,
             marker=markers[index], markeredgecolor=colors[index], markeredgewidth=1.2, markersize=5)
    plt.xlim([0, 600])
    plt.xticks(x)    
    plt.xlabel('Number of clients')
    plt.ylim([0, 0.13])
    plt.yticks(np.arange(0, 0.11, 0.02))
    plt.ylabel('Time (ms)')
    plt.title('(e) Forward response', fontsize=11)

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=3, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()

plt.subplot(2, 3, 6)
for index in range(0, len(labels), 1):
    y = [client_percentiles[index] for client_percentiles in thread_percentiles['total']]
    plt.plot(x, y,
             alpha=0.7, color=colors[index], label=labels[index],
             linestyle='-', linewidth=1,
             marker=markers[index], markeredgecolor=colors[index], markeredgewidth=1.2, markersize=5)
    plt.xlim([0, 600])
    plt.xticks(x)    
    plt.xlabel('Number of clients')
    plt.ylim([0, 100])
    plt.yticks(np.arange(0, 100, 10))
    plt.ylabel('Time (ms)')
    plt.title('(f) Total', fontsize=11)

ax = plt.gca()
ax.grid(True)

plt.legend(bbox_to_anchor=(0., 1., 1., 0.), loc=9, ncol=2, 
           mode="expand", borderaxespad=0., fontsize='small')
plt.tight_layout()